In [2]:
# 02_train_lstm.ipynb

%pip install scikit-learn


import os
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error

import tensorflow as tf
from tensorflow import keras

import joblib
import json




BASE_DIR = "."
PROCESSED_PATH = os.path.join(BASE_DIR, "data", "processed", "daily_store_sales.csv")
MODELS_DIR = os.path.join(BASE_DIR, "models")
os.makedirs(MODELS_DIR, exist_ok=True)

daily = pd.read_csv(PROCESSED_PATH, parse_dates=["purchase_date"])
daily = daily.sort_values("purchase_date").reset_index(drop=True)

daily.head(), daily.shape


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 27.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.9/28.9 MB 17.9 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


(  purchase_date  total_qty  total_revenue  avg_discount  avg_rating  \
 0    2024-08-06         36        1324.80     43.000000    3.900000   
 1    2024-08-07         44        2412.92     16.700000    3.200000   
 2    2024-08-08         44        5467.44      0.000000    4.000000   
 3    2024-08-10         29         738.05      0.000000    3.700000   
 4    2024-08-12        115       12813.79     14.866667    2.233333   
 
    dayofweek  is_weekend  month  year  
 0          1           0      8  2024  
 1          2           0      8  2024  
 2          3           0      8  2024  
 3          5           1      8  2024  
 4          0           0      8  2024  ,
 (273, 9))

In [3]:
# target ที่จะทำนาย
target_col = "total_qty"

# features รวมฟีเจอร์เวลาบวกตัว target lag (ให้ model เห็น demand ในอดีต)
feature_cols = [
    "total_qty",
    "total_revenue",
    "avg_discount",
    "avg_rating",
    "dayofweek",
    "is_weekend",
    "month",
    "year",
]

data = daily.copy()

X_raw = data[feature_cols].values.astype("float32")
y_raw = data[target_col].values.astype("float32")


In [4]:
WINDOW_SIZE = 30  # ดูย้อนหลัง 30 วัน

scaler_x = MinMaxScaler()
X_scaled = scaler_x.fit_transform(X_raw)

def create_sequences(features, target, window_size):
    X, y = [], []
    for i in range(len(features) - window_size):
        X.append(features[i:i+window_size])
        y.append(target[i+window_size])
    return np.array(X, dtype="float32"), np.array(y, dtype="float32")

X_all, y_all = create_sequences(X_scaled, y_raw, WINDOW_SIZE)
X_all.shape, y_all.shape


((243, 30, 8), (243,))

In [5]:
n_samples = X_all.shape[0]
train_size = int(n_samples * 0.8)  # 80% แรก = train, 20% หลัง = test

X_train, X_test = X_all[:train_size], X_all[train_size:]
y_train, y_test = y_all[:train_size], y_all[train_size:]

X_train.shape, X_test.shape


((194, 30, 8), (49, 30, 8))

In [6]:
n_features = X_train.shape[2]

model = keras.Sequential([
    keras.layers.Input(shape=(WINDOW_SIZE, n_features)),
    keras.layers.LSTM(64, return_sequences=False),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dense(1)  # ทำนาย demand (ชิ้น) เป็นตัวเลขต่อวัน
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss="mae",
    metrics=[keras.metrics.MeanAbsoluteError(name="mae"),
             keras.metrics.MeanAbsolutePercentageError(name="mape")]
)

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        18,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,801 (81.25 KB)

 Trainable params: 20,801 (81.25 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=10,
        restore_best_weights=True
    )
]

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=16,
    callbacks=callbacks,
    verbose=1
)


Epoch 1/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 48.3763 - mae: 48.3763 - mape: 3479270.0000 - val_loss: 877.1595 - val_mae: 877.1595 - val_mape: 53796552.0000
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 45.6068 - mae: 45.6068 - mape: 31673272.0000 - val_loss: 871.7966 - val_mae: 871.7966 - val_mape: 168872768.0000
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 41.9228 - mae: 41.9228 - mape: 91292656.0000 - val_loss: 866.5553 - val_mae: 866.5553 - val_mape: 297978944.0000
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 32.1940 - mae: 32.1940 - mape: 83479200.0000 - val_loss: 862.6649 - val_mae: 862.6649 - val_mape: 423840512.0000
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 35.3351 - mae: 35.3351 - mape: 199045200.0000 - val_loss: 860.0237 - val_mae: 860.0237 - val_mape: 531837728.0000
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 27.8867 - mae: 27.8867 - mape: 424880000.0000 - val_loss: 858.0629 - val_mae: 858

In [8]:
# predict
y_pred = model.predict(X_test).flatten()

mae = mean_absolute_error(y_test, y_pred)

# MAPE แบบกันศูนย์
eps = 1e-8
mape = np.mean(np.abs((y_test - y_pred) / (y_test + eps))) * 100

print(f"Test MAE  : {mae:.2f}")
print(f"Test MAPE : {mape:.2f}%")


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
Test MAE  : 855.11
Test MAPE : 8840873984.00%


In [10]:
# baseline: ใช้ค่าจริงของวันก่อนหน้าเป็นคำทำนาย
y_test_baseline = y_test[:-1]      # ย้อน 1 วัน
y_pred_baseline = y_test[1:]       # ทำนายให้ตรง index

mae_base = mean_absolute_error(y_test[1:], y_test_baseline)
print("Baseline MAE:", mae_base)
print("Model MAE   :", mae)


Baseline MAE: 886.4166870117188
Model MAE   : 855.1065673828125


In [9]:
# 1) Save Keras model
MODEL_PATH = os.path.join(MODELS_DIR, "fashion_lstm.h5")
model.save(MODEL_PATH)
print("Saved model to:", MODEL_PATH)

# 2) Save scaler_x ด้วย pickle
SCALER_PATH = os.path.join(MODELS_DIR, "scaler_x.pkl")
joblib.dump(scaler_x, SCALER_PATH)
print("Saved scaler to:", SCALER_PATH)

# 3) Save config (feature names & window size)
config = {
    "window_size": WINDOW_SIZE,
    "feature_cols": feature_cols,
    "target_col": target_col,
}

CONFIG_PATH = os.path.join(MODELS_DIR, "config.json")
with open(CONFIG_PATH, "w", encoding="utf-8") as f:
    json.dump(config, f, indent=2)
print("Saved config to:", CONFIG_PATH)


Saved model to: ./models/fashion_lstm.h5
Saved scaler to: ./models/scaler_x.pkl
Saved config to: ./models/config.json
